In [42]:
# importa as bibliotecas necessárias
import PyPDF2
import re
import string

In [43]:
# Abre o arquivo pdf 
path = r'C:\Users\MatheusPereira\OneDrive - Pontte\Área de Trabalho\automacaoRegistroCCI\Arquivos Simulação\HE_Contrato_Agostinho_Assinatura Digital_VFinal.pdf'
pdf_file = open(path, 'rb')


In [44]:
#Faz a leitura usando a biblioteca
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto 
text=''
for i in range(0,9):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')

In [45]:
number_of_pages

36

In [46]:
text

"CONTRATO DE EMPRÉSTIMO (MÚTUO) E PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTIA COM EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO - CCI CONTRATO Nº 0001013447/AFD REFIN – SFI RUBRICA DAS PARTES Contrato de Empréstimo (Mútuo) com Alienação Fiduciária – Pessoa Física Página 1 de 36 Pelo presente contrato, celebrado nos termos do artigo 38 da Lei 9.514/97, de acordo com a redação que lhe deu o artigo 57 da Lei 10.931/2004, as partes a seguir enunciadas e qualificadas formalizam este CONTRATO DE EMPRÉSTIMO (MÚTUO) E PACTO ADJETO DE ALIENAÇÃO FIDUCIÁRIA EM GARANTIA COM EMISSÃO DE CÉDULA DE CRÉDITO IMOBILIÁRIO - CCI, tendo por objeto o imóvel identificado no campo 5 a seguir, e consoante cláusulas e condições adiante estipuladas : CAMPO 1 – CREDORA RAZÃO SOCIAL: QI SOCIEDADE DE CRÉDITO DIRETO S.A. ENDEREÇO COMERCIAL: Avenida Brigadeiro Faria Lima, nº 2.391, 1º andar COMPLEMENTO: Conjunto 12 BAIRRO: Jardim Paulistano CIDADE: São Paulo UF: SP CEP: 01452-000 CNPJ: 32.402.502/0001-35 DATA DA CONST

In [47]:
listaDePara = {'valorTotal':'VALOR DO EMPRÉSTIMO: R$','tabela': 'SISTEMA DE AMORTIZAÇÃO:','registro': 'DESPESAS DE REGISTRO: R$','Taxa ao Mes':'H.1. NOMINAL:','valorLiquido': '-J-K-L-M-N): R$',
                'prazoMes': 'PRAZO DE AMORTIZAÇÃO:','valorPrimeiraParcelaComEncargos':'VALOR TOTAL DO PRIMEIRO ENCARGO, NESTA DATA: R$',
                'valorImóvel':'Valor de avaliação do Imóvel para fins de leilão: R$','prazoContrato': 'N.º DE PRESTAÇÕES:','ultimaParcela':'DATA DO TÉRMINO DO PRAZO CONTRATUAL:','dataContrato': 'DATA DE DESEMBOLSO:',
                'valorPrimeiraParcela':'VALOR DA PARCELA MENSAL (AMORTIZAÇÃO E JUROS), NESTA DATA: R$','primeiraParcela':'VENCIMENTO DA PRIMEIRA PRESTAÇÃO :','indice':'ÍNDICE: MENSAL do'
                }

#len(listaHE) # <--- Qtd de Itens na Lista
listaKey = []
listaValues = []

for key, value in listaDePara.items():
    inicioFrase = text.find(value,0)
    finalFrase = inicioFrase + len(value) + 1
    proximoEspaco = text.find(" ", finalFrase)
    valorExtraido = text[finalFrase:proximoEspaco]

    #Ajustar Valores Númericos
    if '.' in valorExtraido:
        valorExtraido = valorExtraido.replace(".", "")
        valorExtraido = valorExtraido.replace(",", ".")
    
    #Ajustar Valores Percentuais
    if '%' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", ".")
        valorExtraido = valorExtraido.replace("%", "")
        valorExtraido = round(float(valorExtraido)/100,4)
    elif 'IPCA' or 'IGPM' in valorExtraido:
        valorExtraido = valorExtraido.replace(",", "")
    
    listaKey.append(key)
    listaValues.append(valorExtraido)

In [48]:
# Extraindo numero da matricula
inicioFrase = text.find('matrícula nº',0)
finalFrase = inicioFrase + len('matrícula nº') + 1
ultimaMatricula = text.find("do", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]
valorExtraido = valorExtraido.replace(",", "")
if len(valorExtraido) > 9:
    valorExtraido = 0
    inicioFrase = text.find('nas matrículas nºs',0)
    finalFrase = inicioFrase + len('nas matrículas nºs') + 1
    ultimaMatricula = text.find("do", finalFrase)
    valorExtraido = text[finalFrase:ultimaMatricula]
    valorExtraido = valorExtraido.replace(",", "")
    valorExtraido = valorExtraido.replace("e ", "")

listaKey.append('Matrícula')
listaValues.append(valorExtraido)


In [49]:
# Extraindo cartório
inicioFrase = text.find(' do Livro N°',0)
finalFrase = inicioFrase + len(' do Livro N°') + 1
ultimaMatricula = text.find(".", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]

listaKey.append('Cartório')
listaValues.append(valorExtraido)

In [50]:
# Extraindo participantes da operação 

campo7 = 'CAMPO 7'                         #Definir Variáveis Auxiliares
campo8 = 'CAMPO 8 – CLÁUSULA(S)'

#Pegar posição das variáveis auxiliares no texto
inicioTopico = text.find(campo7, 0)
finalTopico = text.find(campo8, 0)

#Criar Paragráfo Auxiliar
paragrafoAux = text[inicioTopico+len(campo7)+1:finalTopico-1]
paragrafoAux = re.sub('\s+',' ', paragrafoAux)
print(paragrafoAux)

participantesKey = []
participantesValues = []

– COMPOSIÇÃO DA RENDA PARA FINS DE INDENIZAÇÃO NOME: AGOSTINHO FRANCISCO DE SOUZA PARTICIPAÇÃO: 100%


In [51]:


totalParticipantes = paragrafoAux.count('NOME: ')
# Extraindo participantes
fimValor = 0

for num in range(0, totalParticipantes):
    paragrafoAux = paragrafoAux[fimValor:]
    inicioFrase = paragrafoAux.find('NOME: ',0)
    finalFrase = inicioFrase + len('NOME: ') 
    ultimoNome = paragrafoAux.find("PARTICIPAÇÃO: ", finalFrase)
    nomeExtraido = paragrafoAux[finalFrase:ultimoNome]
    participantesKey.append(f'Participante{num+1}')
    participantesKey.append(f'Participação{num+1}')
    participantesValues.append(nomeExtraido)
    inicioFrase = paragrafoAux.find('PARTICIPAÇÃO:',0)
    finalFrase = inicioFrase + len('PARTICIPAÇÃO:') 
    fimValor = paragrafoAux.find("%", finalFrase)
    participacaoExtraido = paragrafoAux[finalFrase:fimValor+1]
    participantesValues.append(participacaoExtraido)
    dict_participantes = dict(zip(participantesKey,participantesValues))
    dict_participantes

In [52]:
# Extraindo CCI
inicioFrase = text.find('NÚMERO: ',0)
finalFrase = inicioFrase + len('NÚMERO: ')
ultimaMatricula = text.find(" ", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]
valorExtraido
listaKey.append('CCI')
listaValues.append(valorExtraido)
print(valorExtraido)

0001013447/AFD


In [53]:
# Extraindo titular
# Definir Variáveis Auxiliares
campo2 = 'CAMPO 2 -'
campo3 = 'CAMPO 3 -'
operacao = 'PF'
# Pegar posição das variáveis auxiliares no texto
inicioTopico = text.find(campo2, 0)
finalTopico = text.find(campo3, 0)

# Criar Paragráfo Auxiliar (Somente com os sub itens do tópico 4. Despesas)
campoTitular= text[inicioTopico+len(campo2)+1:finalTopico-1]
campoTitular = re.sub('\s+',' ', campoTitular)
if 'RAZÃO SOCIAL' in campoTitular:
    operacao = 'PJ'
    inicioFrase = campoTitular.find('RAZÃO SOCIAL: ',0)
    finalFrase = inicioFrase + len('RAZÃO SOCIAL: ')
    fimTitular = campoTitular.find("ENDEREÇO", finalFrase)
    valorExtraido = campoTitular[finalFrase:fimTitular]
    valorExtraido
    listaKey.append('Titular')
    listaValues.append(valorExtraido)
elif 'FIDUCIANTE NOME:':
    operacao = 'PF'
    inicioFrase = campoTitular.find('FIDUCIANTE NOME: ',0)
    finalFrase = inicioFrase + len('FIDUCIANTE NOME: ')
    fimTitular = campoTitular.find("CPF", finalFrase)
    valorExtraido = campoTitular[finalFrase:fimTitular]
    valorExtraido
    listaKey.append('Titular')
    listaValues.append(valorExtraido)



In [54]:
#Criar Dicionario das duas Listas
dict_keyValue = dict(zip(listaKey,listaValues))
dict_keyValue.update(dict_participantes)

In [55]:
# Extraindo data do contrato


text=''

#Ler Página PDF
pageObj = read_pdf.getPage(number_of_pages-2)
#Extrair Texto
text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')

# Extraindo CCI
inicioFrase = text.find('SÃO PAULO, ',0)
finalFrase = inicioFrase + len('SÃO PAULO, ')
ultimaMatricula = text.find(".", finalFrase)
valorExtraido = text[finalFrase:ultimaMatricula]
valorExtraido = valorExtraido.replace(' DE ','/').strip()
if 'JANEIRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('JANEIRO','01')
elif 'FEVEREIRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('FEVEREIRO','02')
elif 'MARÇO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('MARÇO','03')
elif 'ABRIL' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('ABRIL','04')
elif 'MAIO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('MAIO','05')
elif 'JUNHO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('JUNHO','06')
elif 'JULHO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('JULHO','07')
elif 'AGOSTO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('AGOSTO','08')
elif 'SETEMBRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('SETEMBRO','09')
elif 'OUTUBRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('OUTUBRO','10')
elif 'NOVEMBRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('NOVEMBRO','11')
elif 'DEZEMBRO' in valorExtraido:
    dict_keyValue['dataContrato'] = valorExtraido.replace('DEZEMBRO','12')




In [56]:
dict_keyValue

{'valorTotal': '185236.16',
 'tabela': 'SAC',
 'registro': '3042.67',
 'Taxa ao Mes': 0.1162,
 'valorLiquido': '169200.00',
 'prazoMes': '240',
 'valorPrimeiraParcelaComEncargos': '2754.90',
 'valorImóvel': '376000.00',
 'prazoContrato': '234',
 'ultimaParcela': '30/06/2042',
 'dataContrato': '06/06/2022',
 'valorPrimeiraParcela': '2662.09',
 'primeiraParcela': '30/01/2023',
 'indice': 'IPCA',
 'Matrícula': '141.951 ',
 'Cartório': '02 de Registro Geral do 3º Oficial de Registro de Imóveis da Comarca de São Paulo/SP',
 'CCI': '0001013447/AFD',
 'Titular': 'EVEDORES FIDUCIANTES NOME: AGOSTINHO FRANCISCO DE SOUZA ',
 'Participante1': 'AGOSTINHO FRANCISCO DE SOUZA ',
 'Participação1': ' 100%'}

## Def de contagem de participantes

In [14]:
#Faz a leitura usando a biblioteca
pdf_file = open(path, 'rb')
read_pdf = PyPDF2.PdfFileReader(pdf_file)

# pega o numero de páginas
number_of_pages = read_pdf.getNumPages()

#Extriar Texto Página 1 a 5
text=''
for i in range(0,9):
    #Ler Página PDF
    pageObj = read_pdf.getPage(i)
    #Extrair Texto
    text=text+pageObj.extractText()

#Tratar Texto (Remover Quebra de Linhas e espaços duplos)
text = re.sub('\r', '', text) 
text = re.sub('\n', '', text)
text = re.sub(' {2,}', ' ', text).strip(' ')

# Extraindo participantes da operação 
campo7 = 'CAMPO 7'                         #inicio e fim da extração
campo8 = 'CAMPO 8 – CLÁUSULA(S)'

#Pegar posição das variáveis auxiliares no texto
inicioTopico = text.find(campo7, 0)
finalTopico = text.find(campo8, 0)

#Criar Paragráfo Auxiliar
paragrafoAux = text[inicioTopico+len(campo7)+1:finalTopico-1]
paragrafoAux = re.sub('\s+',' ', paragrafoAux)
totalParticipantes = paragrafoAux.count('NOME: ')

paragrafoAux

'– COMPOSIÇÃO DA RENDA PARA FINS DE INDENIZAÇÃO NOME: ALENCAR LUIZ OLIBONI PARTICIPAÇÃO: 33% NOME: LEONARDO COLODA OLIBONI PARTICIPAÇÃO: 33% NOME: MARINA COLODA OLIBONI PARTICIPAÇÃO: 33%'